In [53]:
import pandas as pd
import re
from datetime import datetime
import os

In [54]:
# Regex pattern to parse FIX messages
fix_pattern = r'\|([A-Za-z0-9]+)=([^\|]+)'

In [55]:
def parse_fix_logs(log_file):
    """Extract execution times and prices from FIX logs"""
    pattern = r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}),\d{3}.*\|35=8.*\|55=([A-Z\.]+)\|.*\|31=([\d\.]+)\|.*\|6007=([a-z_]+)'

    data = []
    with open(log_file) as f:
        for line in f:
            match = re.search(pattern, line)
            if match:
                timestamp = datetime.strptime(match.group(1), "%Y-%m-%d %H:%M:%S")
                data.append({
                    'symbol': match.group(2),
                    'price': float(match.group(3)),
                    'strategy': match.group(4),
                    'timestamp': timestamp
                })
    return pd.DataFrame(data)

In [56]:
print("Current directory:", os.getcwd())

Current directory: /Users/mcfeely/Projects/OrderBookHFT/Notebooks


In [57]:
market_maker = parse_fix_logs("/Users/mcfeely/Projects/OrderBookHFT/logs/fix_market_maker.log")
momentum = parse_fix_logs("/Users/mcfeely/Projects/OrderBookHFT/logs/fix_momentum.log")
my_strategy = parse_fix_logs("/Users/mcfeely/Projects/OrderBookHFT/logs/fix_my_strategy.log")
passive = parse_fix_logs("/Users/mcfeely/Projects/OrderBookHFT/logs/fix_passive_liquidity_provider.log")

In [58]:
all_trades = pd.concat([market_maker, momentum, my_strategy, passive])

In [59]:
def find_competitions(df):
    # Group by symbol and price
    grouped = df.groupby(['symbol', 'price'])

    competitions = []
    for (symbol, price), group in grouped:
        if len(group) > 1:  # Multiple strategies at same price
            # Filter trades within 1 second of each other
            group = group.sort_values('timestamp')
            time_diff = group['timestamp'].diff().dt.total_seconds().abs()
            cluster = (time_diff <= 1).cumsum()

            for _, cluster_group in group.groupby(cluster):
                if len(cluster_group) > 1:
                    competitions.append(cluster_group)

    return pd.concat(competitions) if competitions else pd.DataFrame()

competing_trades = find_competitions(all_trades)

KeyError: 'symbol'

In [60]:
def calculate_speed_metrics(competing_trades):
    results = []
    for (symbol, price), group in competing_trades.groupby(['symbol', 'price']):
        first_trade = group.nsmallest(1, 'timestamp')
        results.append({
            'symbol': symbol,
            'price': price,
            'fastest_strategy': first_trade['strategy'].values[0],
            'time_advantage_ms': (group['timestamp'].min() - group['timestamp'].max()).total_seconds() * 1000
        })
    return pd.DataFrame(results)

speed_results = calculate_speed_metrics(competing_trades)

In [61]:
print(speed_results)

     symbol       price            fastest_strategy  time_advantage_ms
0     AD.AS   36.956701                market_maker          -512000.0
1     AD.AS   36.994031  passive_liquidity_provider          -512000.0
2     AD.AS   37.031361  passive_liquidity_provider          -512000.0
3     AD.AS   37.068691                market_maker          -518000.0
4     AD.AS   37.106021  passive_liquidity_provider          -513000.0
5     AD.AS   37.143351                market_maker          -548000.0
6     AD.AS   37.180681                market_maker          -553000.0
7     AD.AS   37.218011                market_maker          -553000.0
8     AD.AS   37.255341                market_maker          -910000.0
9     AD.AS   37.292671                    momentum         -1237000.0
10  ASML.AS  655.776024                market_maker          -513000.0
11  ASML.AS  656.438424  passive_liquidity_provider          -513000.0
12  ASML.AS  657.100824  passive_liquidity_provider          -513000.0
13  AS